<a href="https://colab.research.google.com/github/neharikasrivastav/Twitter_Hack_QstionGen/blob/main/Twitter_hack_QstionGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk.data
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
nlp = spacy.load('en_core_web_sm')
stemmer = LancasterStemmer()

In [3]:
sentences = []
disc_sentences = {}
nondisc_sentences = []
aux_list = ['am', 'are', 'is', 'was', 'were', 'can', 'could', 'does', 'do', 'did', 'has', 'had', 'may', 'might', 'must',
            'need', 'ought', 'shall', 'should', 'will', 'would']
discourse_markers = ['because', 'as a result', 'since', 'when', 'although', 'for example', 'for instance']
qtype = {'because': ['Why'], 'since': ['When', 'Why'], 'when': ['When'], 'although': ['Yes/No'], 'as a result': ['Why'], 
        'for example': ['Give an example where'], 'for instance': ['Give an instance where'], 'to': ['Why']}
target_arg = {'because': 1, 'since': 1, 'when': 1, 'although': 1, 'as a result': 2, 'for example': 1, 'for instance': 1, 
              'to': 1}

In [4]:
from google.colab import files
files.upload()

Saving neha.txt to neha.txt


{'neha.txt': b'The FBI has launched an investigation after hackers hijacked Twitter accounts of a number of high-profile US figures in an apparent Bitcoin scam.\r\n\r\n"The accounts appear to have been compromised" to perpetrate cryptocurrency fraud, said the bureau, urging the public to be vigilant.\r\n\r\nElon Musk, Bill Gates and Joe Biden were among those hit in what Twitter said was a "co-ordinated" attack.\r\n\r\nTheir official accounts requested donations in the cryptocurrency.\r\n\r\n"Everyone is asking me to give back," said a tweet from the account of Mr Gates, the Microsoft founder. "You send $1,000, I send you back $2,000."\r\n\r\nThe US Senate Commerce committee has demanded Twitter brief it about Wednesday\'s incident by 23 July.\r\n\r\nTwitter said the hackers had targeted its employees "with access to internal systems and tools".\r\n\r\n"We know they [the hackers] used this access to take control of many highly-visible (including verified) accounts and Tweet on their be

In [5]:
def sentensify():
    global sentences
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    fp = open('neha.txt')
    data = fp.read()
    sentences = tokenizer.tokenize(data)
    discourse()

In [6]:
def generate_question(question_part, type):

    
    question_part = question_part[0].lower() + question_part[1:]
    if(question_part[-1] == '.' or question_part[-1] == ','):
        question_part = question_part[:-1]
           
    for i in range(0, len(question_part)):
        if(question_part[i] == 'i'):
            if((i == 0 and question_part[i+1] == ' ') or (question_part[i-1] == ' ' and question_part[i+1] == ' ')):
                question_part = question_part[:i] + 'I' + question_part[i + 1: ]
                
    question = ""
    if(type == 'Give an example where' or type == 'Give an instance where'):
        question = type + " " + question_part + '?'
        return question

    aux_verb = False
    res = None
    for i in range(len(aux_list)):
        if(aux_list[i] in question_part.split()):
            aux_verb = True
            pos = i
            break
    if(aux_verb):
        
        
        text = nltk.word_tokenize(question_part)
        tags = nltk.pos_tag(text)
        question_part = ""
        fP = False
        
        for word, tag in tags:
            if(word in ['I', 'We', 'we']):
                question_part += 'you' + " "
                fP = True
                continue
            question_part += word + " "

        
        question = question_part.split(" " + aux_list[pos])
        if(fP):
             question = ["were "] + question
        else:
            question = [aux_list[pos] + " "] + question

        
        if(type == 'Yes/No'):
            question += ['?']
            
        elif(type != "non_disc"):
            question = [type + " "] + question + ["?"]
            
        else:
            question = question + ["?"]
         
        question = ''.join(question)

    
    else:
        aux = None
        text = nltk.word_tokenize(question_part)
        tags = nltk.pos_tag(text)
        comb = ""

        
        
        for tag in tags:
            if(comb == ""):
                if(tag[1] == 'NN' or tag[1] == 'NNP'):
                    comb = 'NN'

                elif(tag[1] == 'NNS' or tag[1] == 'NNPS'):
                    comb = 'NNS'

                elif(tag[1] == 'PRP'):
                    if tag[0] in ['He','She','It']:
                        comb = 'PRPS'
                    else:
                        comb = 'PRPP'
                        tmp = question_part.split(" ")
                        tmp = tmp[1: ]
                        if(tag[0] in ['I', 'we', 'We']):
                            question_part = 'you ' + ' '.join(tmp)
                            
            if(res == None):
                res = re.match(r"VB*", tag[1])
                if(res):
                    
                   
                    question_part = question_part.replace(tag[0], stemmer.stem(tag[0]))
                res = re.match(r"VBN", tag[1])
                res = re.match(r"VBD", tag[1])

        if(comb == 'NN'):
            aux = 'does'
            
        elif(comb == 'NNS'):
            aux = 'do'
            
        elif(comb == 'PRPS'):
            aux = 'does'
            
        elif(comb == 'PRPP'):
            aux = 'do'
            
        if(res and res.group() in ['VBD', 'VBN']):
            aux = 'did'

        if(aux):
            if(type == "non_disc" or type == "Yes/No"):
                question = aux + " " + question_part + "?"

            else:
                question = type + " " + aux + " " + question_part + "?"
    if(question != ""):
        question = question[0].upper() + question[1:]
    return question

In [7]:
def get_named_entities(sent):
    doc = nlp(sent)
    named_entities = [(X.text, X.label_) for X in doc.ents]
    return named_entities

In [8]:
def get_wh_word(entity, sent):
    wh_word = ""
    if entity[1] in ['TIME', 'DATE']:
        wh_word = 'When'
        
    elif entity[1] == ['PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']:
        wh_word = 'What'
        
    elif entity[1] in ['PERSON']:
            wh_word = 'Who'
            
    elif entity[1] in ['NORP', 'FAC' ,'ORG', 'GPE', 'LOC']:
        index = sent.find(entity[0])
        if index == 0:
            wh_word = "Who"
            
        else:
            wh_word = "Where"
            
    else:
        wh_word = "Where"
    return wh_word

In [9]:
def generate_one_word_questions(sent):
    
    named_entities = get_named_entities(sent)
    questions = []
    
    if not named_entities:
        return questions
    
    for entity in named_entities:
        wh_word = get_wh_word(entity, sent)
        
        if(sent[-1] == '.'):
            sent = sent[:-1]
        
        if sent.find(entity[0]) == 0:
            questions.append(sent.replace(entity[0],wh_word) + '?')
            continue
       
        question = ""
        aux_verb = False
        res = None

        for i in range(len(aux_list)):
            if(aux_list[i] in sent.split()):
                aux_verb = True
                pos = i
                break
            
        if not aux_verb:
            pos = 9
        
        text = nltk.word_tokenize(sent)
        tags = nltk.pos_tag(text)
        question_part = ""
        
        if wh_word == 'When':
            word_list = sent.split(entity[0])[0].split()
            if word_list[-1] in ['in', 'at', 'on']:
                question_part = " ".join(word_list[:-1])
            else:
                question_part = " ".join(word_list)
            
            qp_text = nltk.word_tokenize(question_part)
            qp_tags = nltk.pos_tag(qp_text)
            
            question_part = ""
            
            for i, grp in enumerate(qp_tags):
                word = grp[0]
                tag = grp[1]
                if(re.match("VB*", tag) and word not in aux_list):
                    question_part += WordNetLemmatizer().lemmatize(word,'v') + " "
                else:
                    question_part += word + " "
                
            if question_part[-1] == ' ':
                question_part = question_part[:-1]
        
        else:
            for i, grp in enumerate(tags):
                
                
                word = grp[0]
                tag = grp[1]

                if(re.match("VB*", tag) and word not in aux_list):
                    question_part += word

                    if i<len(tags) and 'NN' not in tags[i+1][1] and wh_word != 'When':
                        question_part += " "+ tags[i+1][0]

                    break
                question_part += word + " "
        question = question_part.split(" "+ aux_list[pos])
        question = [aux_list[pos] + " "] + question
        question = [wh_word+ " "] + question + ["?"]
        question = ''.join(question)
        questions.append(question)
    
    return questions        

In [10]:
def discourse():
    temp = []
    target = ""
    questions = []
    global disc_sentences
    disc_sentences = {}
    for i in range(len(sentences)):
        maxLen = 9999999
        val = -1
        for j in discourse_markers:
            tmp = len(sentences[i].split(j)[0].split(' '))  
            
             
            if(len(sentences[i].split(j)) > 1 and tmp >= 3 and tmp < maxLen):
                maxLen = tmp
                val = j
                
        if(val != -1):

          
            if(disc_sentences.get(val, 'empty') == 'empty'):
                disc_sentences[val] = []
                
            disc_sentences[val].append(sentences[i])
            temp.append(sentences[i])


    nondisc_sentences = list(set(sentences) - set(temp))
    
    t = []
    for k, v in disc_sentences.items():
        for val in range(len(v)):
            
            
            question_part = disc_sentences[k][val].split(k)[target_arg[k] - 1]
            q = generate_question(question_part, qtype[k][0])
            if(q != ""):
                questions.append([disc_sentences[k][val],q])
                
                
    for question_part in nondisc_sentences:
        s = "non_disc"
        sentence = question_part
        text = nltk.word_tokenize(question_part)
        if(text[0] == 'Yes'):
            question_part = question_part[5:]
            s = "Yes/No"
            
        elif(text[0] == 'No'):
            question_part = question_part[4:]
            s = "Yes/No"
            
        q = generate_question(question_part, s)
        if(q != ""):
            questions.append([sentence,q])
        l = generate_one_word_questions(question_part)
        questions += [[sentence,i] for i in l]
    print(len(questions))
    
    for pair in questions:
        print("S: ",pair[0])
        print("Q: ",pair[1])
        print()

In [11]:
syntactic_score = [0,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,0,1,0,1,1,1,1,1,1,0,0,
                    1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,1]
fluency_score   = [0,0,1,1,1,0,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,
                    1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,1,0,1,1,0,1,1,1,0,1,1,1,0,0,0,0,0,1,1]
print(len(syntactic_score))
print(sum(syntactic_score)/len(syntactic_score))
print(sum(fluency_score)/sum(syntactic_score))

92
0.7608695652173914
0.8428571428571429


In [12]:
sentensify()

126
S:  "I'm feeling generous because of Covid-19," the tweet added, along with a Bitcoin link address.
Q:  Why do you feel generous ?

S:  The description of the profile read "It was us", alongside a slightly smiling face emoticon.
Q:  Was the description of the profile read `` It us '' , alongside a slightly smiling face emoticon ?

S:  The chair of the Senate Commerce committee has also been in contact with Twitter.
Q:  Has the chair of the Senate Commerce committee also been in contact with Twitter ?

S:  The chair of the Senate Commerce committee has also been in contact with Twitter.
Q:  Where has The chair of the Senate Commerce committee also been in?

S:  The chair of the Senate Commerce committee has also been in contact with Twitter.
Q:  Who has The chair of the Senate Commerce committee also been in?

S:  "We would urge people to treat requests for money or sensitive information on social media with extreme caution," it said in a statement.
Q:  Were `` you urge people to tr